## K-ium

Vital team

#### Import package

In [ ]:
from google import colab
colab.drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/Kium/test_set/test_set/

import evaluate
from evaluate import calc_accuracy

/content/drive/MyDrive/Colab Notebooks/Kium/test_set/test_set


In [ ]:
! pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 76.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.6 MB/s eta 0:00:00


In [ ]:
import warnings
warnings.filterwarnings('ignore')

from glob import glob
import pandas as pd
import numpy as np
from tqdm import tqdm
#import cv2

import os
import timm
import random
import gc

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim
import collections
from sklearn.metrics import roc_auc_score, accuracy_score
import time

from sklearn.model_selection import StratifiedKFold
from PIL import Image
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split


device = torch.device('cuda:0')
print(torch.cuda.is_available())

True


#### Data preparation

In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/Kium/train_set/train_set/'
save_path = '//content/drive/MyDrive/Colab Notebooks/Kium/train_set/'

Train set : 8616(1077 x 8), test set : 400(50 x 8)

In [ ]:
label = pd.read_csv(path +'train.csv')
label.corr().Aneurysm

Index       -0.061326
Aneurysm     1.000000
L_ICA        0.335338
R_ICA        0.335338
L_PCOM       0.192393
R_PCOM       0.189538
L_AntChor    0.177715
R_AntChor    0.136911
L_ACA        0.148081
R_ACA        0.116091
L_ACOM       0.268029
R_ACOM       0.177715
L_MCA        0.270179
R_MCA        0.327925
L_VA         0.132994
R_VA         0.106692
L_PICA       0.055519
R_PICA       0.045311
L_SCA        0.078621
R_SCA        0.064137
BA           0.158521
L_PCA        0.055519
R_PCA        0.032026
Name: Aneurysm, dtype: float64

In [ ]:
dir = sorted(os.listdir(path))[:-1]

In [ ]:
len(dir)

9016

In [ ]:
crop1000 = transforms.Compose(
                [
                    transforms.CenterCrop(size=(1000,1000)),
                ])

In [ ]:
test_img = []
for i in range(400):
    raw_image_test=Image.open(path+str(dir[i]))
    #raw_image_test=temp.copy()
    if raw_image_test.size[1]==1000:
        crop1000(raw_image_test)

    test_img.append(raw_image_test)
    #temp.close()

print(len(test_img)) #400
print(np.array(test_img[0]).shape)

400
(720, 720, 3)


In [ ]:
from torchvision.transforms.functional import to_pil_image
test_image = []

tot =0
num =0
for i in range(len(test_img)//8):
    temp_LV = None
    temp_RV = None
    tot = tot+1
    for j in range(8):
        temp = test_img[i * 8 + j]
        width, height = temp.size # width, height

        if j%8==0: #LI-A
            temp = temp.crop((width*0.3,height*0.1,width*0.98,height*0.98))#  (left,up, right, down)
        elif j%8==1:  #LI-B
            temp = temp.crop((width*0.02,height*0.02,width*0.98,height*0.98))
        elif j%8==2:  #LV-A
            temp = temp.crop((width*0.15,height*0.02,width*0.85,height*0.98))
        elif j%8==3:  #LV-B
            temp = temp.crop((width*0.02,height*0.1,width*0.98,height*0.98))
        elif j%8==4:  #RI-A
            temp = temp.crop((width*0.02,height*0.1,width*0.7,height*0.98))
        elif j%8==5:  #RI-B
            temp = temp.crop((width*0.02,height*0.02,width*0.98,height*0.98))
        elif j%8==6:  #RV-A
            temp = temp.crop((width*0.15,height*0.02,width*0.85,height*0.98))
        elif j%8==7:  #RV-B
            temp = temp.crop((width*0.02,height*0.1,width*0.98,height*0.98))
        else:
            num=num+1

        test_image.append(temp)

print(len(test_image))

400


In [ ]:
train_img= []

for i in range(8616):
    if '2417' in str(dir[i+400]): continue
    raw_image_train=Image.open(path+str(dir[i+400]))
    #raw_image_train = temp.copy()
    if raw_image_train.size[1]==1000: # 2000번대 데이터중 이미지 주변의 어두운 픽셀과 글자를 제거하기 위함
        raw_image_train = crop1000(raw_image_train)

    train_img.append(raw_image_train)
    #temp.close()
print(len(train_img))

8608


In [ ]:
from torchvision.transforms.functional import to_pil_image
train_image = []

tot =0
num =0
for i in range(len(train_img)//8):
    temp_LV = None
    temp_RV = None
    tot = tot+1
    for j in range(8):
        temp = train_img[i * 8 + j]
        width, height = temp.size # width, height

        if j%8==0: #LI-A
            temp = temp.crop((width*0.3,height*0.1,width*0.98,height*0.98))#  (left,up, right, down)
        elif j%8==1:  #LI-B
            temp = temp.crop((width*0.02,height*0.02,width*0.98,height*0.98))
        elif j%8==2:  #LV-A
            temp = temp.crop((width*0.15,height*0.02,width*0.85,height*0.98))
        elif j%8==3:  #LV-B
            temp = temp.crop((width*0.02,height*0.1,width*0.98,height*0.98))
        elif j%8==4:  #RI-A
            temp = temp.crop((width*0.02,height*0.1,width*0.7,height*0.98))
        elif j%8==5:  #RI-B
            temp = temp.crop((width*0.02,height*0.02,width*0.98,height*0.98))
        elif j%8==6:  #RV-A
            temp = temp.crop((width*0.15,height*0.02,width*0.85,height*0.98))
        elif j%8==7:  #RV-B
            temp = temp.crop((width*0.02,height*0.1,width*0.98,height*0.98))
        else:
            num=num+1

        train_image.append(temp)
print(len(train_image))

8608


In [ ]:
label["IL_Aneurysm"] = (label["L_ICA"] + label["L_PCOM"] +label["L_AntChor"]+ label["L_ACA"]+label["L_ACOM"]+ label["L_MCA"])
label["VL_Aneurysm"] = (label["L_VA"]+ label["L_PICA"]+ label["L_SCA"]+label["BA"] + label["L_PCA"])
#print("단순합 : ", sum(label["L_Aneurysm"]))
label["IL_Aneurysm"]= (label["IL_Aneurysm"]>0).astype(int)
label["VL_Aneurysm"]= (label["VL_Aneurysm"]>0).astype(int)
print("Sum of IL : ",sum(label["IL_Aneurysm"]))
print("Sum of VL : ",sum(label["VL_Aneurysm"]))

Sum of IL :  287
Sum of VL :  50


In [ ]:
label["IR_Aneurysm"] = (label["R_ICA"] + label["R_PCOM"] +label["R_AntChor"]+ label["R_ACA"]+label["R_ACOM"]+ label["R_MCA"])
label["VR_Aneurysm"] = (label["R_VA"]+ label["R_PICA"]+ label["R_SCA"]+label["BA"] + label["R_PCA"])
#print("단순합 : ", sum(label["L_Aneurysm"]))
label["IR_Aneurysm"]= (label["IR_Aneurysm"]>0).astype(int)
label["VR_Aneurysm"]= (label["VR_Aneurysm"]>0).astype(int)
print("Sum of IR : ",sum(label["IR_Aneurysm"]))
print("Sum of VR : ",sum(label["VR_Aneurysm"]))

Sum of IR :  265
Sum of VR :  41


In [ ]:
train_label = label.iloc[50:]
train_label.drop(943, axis = 0, inplace = True) #2417 제거
test_label = label.iloc[:50]

train_a_LI = np.array(train_label.IL_Aneurysm)
train_a_LV = np.array(train_label.VL_Aneurysm)
train_a_RI = np.array(train_label.IR_Aneurysm)
train_a_RV = np.array(train_label.VR_Aneurysm)

train_a_concat = np.concatenate([[train_a_LI],[train_a_LI],[train_a_LV],[train_a_LV],[train_a_RI],[train_a_RI],[train_a_RV],[train_a_RV]], axis = 0).transpose()  # 지역별 인당 이미지가 4개 -> label 복제
train_aneurysm = train_a_concat.reshape(-1,)

test_a = np.array(test_label.Aneurysm)

train_aneurysm.shape, test_a.shape

((8608,), (50,))

### 1. Aneurysm prediction

#### Define custom dataset and pretrained model(ResNet)

In [ ]:
class Custom_dataset(Dataset):
    def __init__(self, img_paths, labels, mode='train'):
        self.img_paths = img_paths
        self.labels = labels
        self.mode=mode
    def __len__(self):
        return len(self.img_paths)
    def __getitem__(self, idx):
        img = self.img_paths[idx]
        if self.mode == 'train':
            train_transform = transforms.Compose([
                #transforms.Grayscale(),
                transforms.RandomRotation((-45, 45)),
                transforms.RandomHorizontalFlip(p=0.2),
                transforms.RandomVerticalFlip(p=0.2),

                transforms.ToTensor(),
                transforms.Resize(size = (768, 768)),
                #transforms.Normalize([183, 183, 183], [29, 29, 29])
            ])
            img = train_transform(img)


        if self.mode == 'test':
            test_transform = transforms.Compose([
                #transforms.Grayscale(),
                transforms.ToTensor(),
                transforms.Resize(size = (768, 768)),
                #transforms.Normalize([183, 183, 183], [29, 29, 29])
            ])
            img = test_transform(img)



        label = self.labels[idx]
        return img, label

class Network_for_Aneurysm(nn.Module):
    def __init__(self,mode = 'train'):
        super(Network_for_Aneurysm, self).__init__()
        self.mode = mode
        if self.mode == 'train':
            self.model = timm.create_model('resnet50',pretrained=True, num_classes=1, drop_path_rate = 0.2, in_chans = 3) # pretrained_model : Resnet
        if self.mode == 'test':
            self.model = timm.create_model('resnet50',pretrained=True, num_classes=1, drop_path_rate = 0, in_chans = 3)
        self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.model(x)
        x = self.activation(x)
        return x


In [ ]:
def score_function(real, pred):
    score = roc_auc_score(real, pred)
    return score

def score_function2(real, pred):
    score = accuracy_score(real, pred)
    return score

In [ ]:
def main(seed = 2023):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = True

main(2023)

#### Aneurysm_Training

5-fold

In [ ]:
from sklearn.model_selection import StratifiedKFold

cv = StratifiedKFold(n_splits = 5, random_state = 2023, shuffle=True)
batch_size = 32
epochs = 40
pred_ensemble = []
best_epoch_list = [0,0,0,0,0]

for idx, (train_idx, val_idx) in enumerate(cv.split(train_image, train_aneurysm)): # It is designed to learn each model in each fold.
    if idx == 0 : continue

    print("----------fold_{} start!----------".format(idx+1))
    t_imgs, val_imgs = np.array(train_img)[train_idx],  np.array(train_img)[val_idx]
    t_labels, val_labels = np.array(train_aneurysm)[train_idx], np.array(train_aneurysm)[val_idx]

    # Train
    train_dataset = Custom_dataset(np.array(t_imgs), np.array(t_labels), mode='train')
    train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)

  # Val
    val_dataset = Custom_dataset(np.array(val_imgs), np.array(val_labels), mode='test')
    val_loader = DataLoader(val_dataset, shuffle=True, batch_size= batch_size)

    gc.collect()
    torch.cuda.empty_cache()

    model = Network_for_Aneurysm().to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr = 2e-4)
    criterion = nn.BCELoss()
    scaler = torch.cuda.amp.GradScaler()

    best_roc = 0
    early_stopping = 0

    for epoch in range(epochs):
        start=time.time()
        train_loss = 0
        train_pred=[]
        train_label=[]
        model.train()
        for batch in (train_loader):
            optimizer.zero_grad()
            x = torch.tensor(batch[0], dtype=torch.float32, device=device)
            y = torch.tensor(batch[1], dtype=torch.float32, device=device)
            with torch.cuda.amp.autocast():
                pred = model(x)
            loss = criterion(pred.reshape(-1).to(torch.float32),y)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            train_loss += loss.item()/len(train_loader)
            train_pred += pred.detach().cpu().numpy().tolist()
            train_label += y.detach().cpu().numpy().tolist()
        train_roc = score_function(train_label, train_pred)
        state_dict= model.state_dict()

        model.eval()
        with torch.no_grad():
            val_loss = 0
            val_pred = []
            val_y = []
            for batch in (val_loader):
                x_val = torch.tensor(batch[0], dtype = torch.float32, device = device)
                y_val = torch.tensor(batch[1], dtype=torch.float32, device=device)
                with torch.cuda.amp.autocast():
                    pred_val = model(x_val)
                loss_val = criterion(pred_val.reshape(-1).to(torch.float32), y_val)

                val_loss += loss_val.item()/len(val_loader)
                val_pred += pred_val.detach().cpu().numpy().tolist()
                val_y += y_val.detach().cpu().numpy().tolist()
            val_roc = score_function(val_y, val_pred)

            if val_roc > best_roc : # If val_roc is larger than best_roc, model's weights are saved.
                best_epoch = epoch
                best_epoch_list[idx] = best_epoch
                best_loss = val_loss
                best_roc = val_roc
                early_stopping = 0

                torch.save({'epoch':epoch,
                      'state_dict':state_dict,
                      'optimizer': optimizer.state_dict(),
                      'scaler': scaler.state_dict(),}
                     , save_path +'best_resnet_fold{}_epoch{}.pth'.format(idx+1,epoch+1))
                print('-----------------SAVE:{} epoch----------------'.format(best_epoch+1))


            else:
                early_stopping += 1

            # Early Stopping
            if early_stopping == 13:
                TIME = time.time() - start
                print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
                print(f'TRAIN    loss : {train_loss:.5f}    roc : {train_roc:.5f}')
                print(f'Val    loss : {val_loss:.5f}    roc : {val_roc:.5f}')
                break

        TIME = time.time() - start
        print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
        print(f'TRAIN    loss : {train_loss:.5f}    roc : {train_roc:.5f}')
        print(f'Val    loss : {val_loss:.5f}    roc : {val_roc:.5f}')


----------fold_2 start!----------


-----------------SAVE:1 epoch----------------
epoch : 1/40    time : 196s/7632s
TRAIN    loss : 0.39838    roc : 0.64992
Val    loss : 0.38322    roc : 0.72655
-----------------SAVE:2 epoch----------------
epoch : 2/40    time : 171s/6485s
TRAIN    loss : 0.37070    roc : 0.71971
Val    loss : 0.42862    roc : 0.73760
-----------------SAVE:3 epoch----------------
epoch : 3/40    time : 172s/6369s
TRAIN    loss : 0.36459    roc : 0.73601
Val    loss : 0.36819    roc : 0.74838
-----------------SAVE:4 epoch----------------
epoch : 4/40    time : 171s/6159s
TRAIN    loss : 0.36040    roc : 0.74836
Val    loss : 0.36022    roc : 0.76817
epoch : 5/40    time : 170s/5942s
TRAIN    loss : 0.35189    roc : 0.76203
Val    loss : 0.37745    roc : 0.72799
-----------------SAVE:6 epoch----------------
epoch : 6/40    time : 173s/5889s
TRAIN    loss : 0.34987    roc : 0.76779
Val    loss : 0.34919    roc : 0.77829
epoch : 7/40    time : 170s/5608s
TRAIN    loss : 0.34781    roc : 0.77411
Val    loss

#### Aneurysm_Test

Ensemble

In [ ]:
best_epochs = [34,19,29,26,29]
pred_prob = []
pred_ensemble = []
pred_ensemble2 = []

batch_size = 8
gc.collect()
torch.cuda.empty_cache()
# Test
test_dataset = Custom_dataset(np.array(test_image), np.array(["tmp"]*len(test_image)), mode='test')
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

for i in range(5): # Ten prediction results were obtained using the ten stored model weights.
    if i == 1 : continue
    print('----------Fold{} prediction start----------'.format(i+1))
    model_test = Network_for_Aneurysm(mode = 'test').to(device)
    model_test.load_state_dict(torch.load((save_path +'best_resnet_fold{}_epoch{}.pth'.format(i+1,best_epochs[i])))['state_dict'])
    model_test.eval()
    pred_A = []
    with torch.no_grad():
        for batch in (test_loader):
            x = torch.tensor(batch[0], dtype = torch.float32, device = device)
            with torch.cuda.amp.autocast():
                pred = model_test(x)
                pred_A.append(pred.detach().cpu().numpy().tolist())
        pred_ensemble.append(np.mean(pred_A, axis = 1))
        pred_ensemble2.append(np.max(pred_A, axis = 1))
    pred_prob.append(pred_A)
    print('Fold{} prediction mean roc: {}'.format(i+1,roc_auc_score(test_label.Aneurysm, np.mean(pred_A, axis = 1))))
    print('Fold{} prediction max roc: {}'.format(i+1,roc_auc_score(test_label.Aneurysm, np.max(pred_A, axis = 1))))


----------Fold1 prediction start----------
Fold1 prediction mean roc: 0.8099838969404187
Fold1 prediction max roc: 0.7906602254428341
----------Fold2 prediction start----------
Fold2 prediction mean roc: 0.7310789049919484
Fold2 prediction max roc: 0.7423510466988728
----------Fold3 prediction start----------
Fold3 prediction mean roc: 0.8051529790660225
Fold3 prediction max roc: 0.7681159420289855
----------Fold4 prediction start----------
Fold4 prediction mean roc: 0.821256038647343
Fold4 prediction max roc: 0.7906602254428341
----------Fold5 prediction start----------
Fold5 prediction mean roc: 0.7954911433172303
Fold5 prediction max roc: 0.7584541062801933


In [ ]:
roc_auc_score(test_label.Aneurysm, np.max(pred_ensemble, axis = 0)), roc_auc_score(test_label.Aneurysm, np.mean(pred_ensemble2, axis = 0))

(0.8099838969404187, 0.8051529790660226)

In [ ]:
auc = roc_auc_score(test_label.Aneurysm, np.max(pred_ensemble, axis = 0))

### Location prediction

In [ ]:
location = test_label.drop(columns = ["Index", 'Aneurysm'])
output_location = location.copy()
output_location[output_location == 1] = 0

LI = np.mean(np.mean(pred_prob, axis = 0)[:, [0,1]], axis = 1)
RI = np.mean(np.mean(pred_prob, axis = 0)[:, [4,5]], axis = 1)

LI_prediction = np.where(LI > 0.55,1,0)
RI_prediction = np.where(RI > 0.55,1,0)

output_location["L_ICA"] = LI_prediction
output_location["R_ICA"] = RI_prediction

In [ ]:
calc_accuracy(location, output_location)

0.9392

In [ ]:
acc = calc_accuracy(location, output_location)

### AUC & ACC

In [ ]:
print('AUROC of the provided model')
print (auc)
print('Accuracy for locations')
print (acc)

AUROC of the provided model
0.8099838969404187
Accuracy for locations
0.9392
